# Work 1 

In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
from base.Splendor_v3.env import *

import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]

print(bias1)

[array([7.06310946e-04, 9.97517502e-02, 2.38999697e-01, 8.57861519e-01,
       7.12358827e-01, 8.47073913e-01, 4.29212562e-01, 2.98309988e-02,
       2.43473023e-01, 2.98752425e-01, 9.39882597e-01, 8.59724947e-01,
       5.11224992e-01, 9.78640887e-01, 6.34574741e-01]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]


In [3]:
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 100000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)


[0.93983327 0.95673182 0.97823742 1.         0.91100257 0.90164033
 0.90086591 0.90437237 0.90438574 0.90444899 0.9109521  0.91056621
 0.91736627 0.90839421 0.9074557 ]


In [24]:
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

5624


#  feature cho các action 0 


In [27]:
#State old là state Hiện tại 
#State new là state Mới 
old = np.load('old0.npy')
new = np.load('new0.npy')
# state mà có cả Value của old and new (1,0)
o = (new>old)
o= 1*o
i = np.zeros((245142,1))
for u in range(245142) :
    if sum(o[u])>0 :
        # print(o[u])
        i[u] = 1
#  a matrix with random value
l = np.random.uniform(-1,1,(161,1))
#state mà kích hoạt (1,0)
k = 1*((new@l)>=0)
# state mà kích hoạt và có reward (1,0)
y = np.zeros((245142,1))
for p in range(245142):
    if sum(k[p]) >= 1 and sum(i[p])>= 1:
        y[p] = 1
    else:
        y[p] = 0
        
print(sum(y)/sum(k))
# print(old.shape)
print(sum(y))
l = l.reshape(1,161)

[0.9995557]
[110238.]


In [28]:
perx=[0]
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = l@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 0 
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p0, 1000,perx,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 23.3


#  feature cho các action 1


In [23]:
old1 = np.load('old1.npy')
new1 = np.load('new1.npy')
# state mà có cả Value của old and new (1,0)
o1 = (new1>old1)
o1= 1*o1
i1 = np.zeros((245378,1))
for u1 in range(245378) :
    if sum(o1[u1])>0 :
        i1[u1] = 1

        
#  a matrix with random value
l1 = np.random.uniform(-1,1,(161,1))

#state mà kích hoạt (1,0)
k1 = 1*((new1@l1)>=0)
# state mà kích hoạt và có reward (1,0)
y1 = np.zeros((245378,1))
for p1 in range(245378):
    if sum(k1[p1]) >= 1 and sum(i1[p1])>= 1:
        y1[p1] = 1
    else:
        y1[p1] = 0
print(sum(y1)/sum(k1))
# print(old.shape)
print(sum(y1))
l1 = l1.reshape(1,161)

[0.9998423]
[190199.]


In [29]:
@njit
def p_action1(state,per):
    actions = getValidActions(state)
    matran1 = l1@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_action1, 1000,perx,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 39.800000000000004


# Ghép 2 action với nhau 

In [28]:
perx1 = [bias_greedy]

0.9504946791405442


In [ ]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = l1@state   
    matran2 = l@state     
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    for i in range(len(matran2)):
        matran2[i] = int(matran2[i] >= 0)

    if matran1 == 1 and matran2 == 0:
        action = 1
    elif matran2 == 1 and matran1 == 0 :
        action = 0
    elif matran2 == matran1 and matran1 == 1:
        if per[0][0]+1 > per[0][1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1 or action == 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 1000,perx1,0)
print('tỷ lệ thắng',(win/1000)*100)

# Homework 1:

*  **1.For each action , save at least 200k state both old and new state** 
*  **2.Label the state  which change 0 and 1** 
*  **3.Predict what value in state will change** 




**1.For each action , save at least 200k state both old and new state** 

In [ ]:
import numpy as np
from base.Splendor_v3.env import *
for k in range(2,16):
  @njit()
  def p0(state,per):
    # Choose an action
    actions = getValidActions(state)
    actions = np.where(actions == 1)[0]
    action = actions[np.random.randint(len(actions))]
    # Lưu new state
    if  per[3][0][0] == 1: 
      if int(per[4][0][0])<=10:
        j = int(per[4][0][0])
        for i in range(getStateSize()):
          per[2][j][i] += state[i]
        print('lưu state sau khi action '+f'{k}',per[2][j])
        # trigger action 2
        per[3][0][0] = 0
        per[4][0][0] += 1
        print("action sau khi action "+f'{k}',action)

    #Lưu old state 
    if action == k :
      if int(per[1][0][0])<=10:
        # save state
        j = int(per[1][0][0])
        for i in range(getStateSize()):
          per[0][j][i] += state[i]
        print('lưu state trước khi action '+f'{k}',per[0][j])
        # count number of state
        per[1][0][0] += 1
        per[3][0][0] = 1
        print("action ",action)
      
    return action,per

          #0                                  #1                #2                              #3            #4
  perx = [np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((1,1))]
            #state old                      #số trận của old       #state new                  # trigger      # số trận của new
  win, x = numba_main_2(p0, 10,perx , 0)
  np.save('old'+f'{k}'+'.npy',x[0])
  np.save('new'+f'{k}'+'.npy',x[2])

In [12]:
old1 = np.load('old1.npy')
new1 = np.load('new1.npy')
# state mà có cả Value của old and new (1,0)
o1 = (new1>old1)
o1= 1*o1
# state mà có reward (1,0)
i1 = np.zeros((245378,1))
for u1 in range(245378) :
    if sum(o1[u1])>0 :
        i1[u1] = 1
print(np.where(i1==0)[0].shape)


(9181,)


*  **2.Label the state  which change 0 and 1** 
*  **3.Predict what value in state will change** 

**Logistic regression**

In [32]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_img, test_img, train_lbl, test_lbl = train_test_split(old, i, test_size=1/7.0, random_state=0)

clf1 = LogisticRegression(random_state=0).fit(train_img, train_lbl)
clf1.score(test_img, test_lbl)


0.9730130655559993

In [26]:
print(clf1.predict(np.array(old1[0]).reshape(1,161)))

[1.]
[1.]


# Homework 2:
**Tìm bộ feature cho từng action sao cho**
* **(tỷ lệ state vừa kích hoạt vừa có value /tỷ lệ state vừa kích hoạt) luôn bằng 1**
* **check xem từng action tỷ lệ thắng có trên 30%**

# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**